1) Mount Google Drive & Import Libraries

In [28]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2) Load Dataset

In [30]:
import pandas as pd

# Load dataset (adjust path to your drive location)
file_path = '/content/drive/MyDrive/ML/spam.csv'
df = pd.read_csv(file_path, encoding='latin-1')

# Keep only required columns
df = df[['v1', 'v2']]
df.columns = ['label', 'text']

print("Dataset Sample:")
print(df.head())

Dataset Sample:
  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


3) Preprocess Text

In [33]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab') # Add this line

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

def preprocess_text(text, use_stemming=False, use_lemmatization=True):
    # 1. Convert to lowercase
    text = text.lower()

    # 2. Remove punctuation & numbers
    text = re.sub(r'[^a-z\s]', '', text)

    # 3. Tokenization
    tokens = word_tokenize(text)

    # 4. Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # 5. Apply stemming or lemmatization
    if use_stemming:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]

    if use_lemmatization:
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return " ".join(tokens)

df['clean_text'] = df['text'].apply(preprocess_text)

print("Original vs Cleaned Text:")
print(df[['text', 'clean_text']].head(10))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Original vs Cleaned Text:
                                                text  \
0  Go until jurong point, crazy.. Available only ...   
1                      Ok lar... Joking wif u oni...   
2  Free entry in 2 a wkly comp to win FA Cup fina...   
3  U dun say so early hor... U c already then say...   
4  Nah I don't think he goes to usf, he lives aro...   
5  FreeMsg Hey there darling it's been 3 week's n...   
6  Even my brother is not like to speak with me. ...   
7  As per your request 'Melle Melle (Oru Minnamin...   
8  WINNER!! As a valued network customer you have...   
9  Had your mobile 11 months or more? U R entitle...   

                                          clean_text  
0  go jurong point crazy available bugis n great ...  
1                            ok lar joking wif u oni  
2  free entry wkly comp win fa cup final tkts st ...  
3                u dun say early hor u c already say  
4           nah dont think go usf life around though  
5  freemsg hey darling week

4) Feature Extraction (TF-IDF)

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Convert cleaned text into TF-IDF features
X = vectorizer.fit_transform(df['clean_text'])
y = df['label']

print("Shape of TF-IDF matrix:", X.shape)

Shape of TF-IDF matrix: (5572, 7855)


5) Split data

In [35]:
from sklearn.model_selection import train_test_split

# Split data into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)


Training set size: (4457, 7855)
Testing set size: (1115, 7855)


6) Train Naive Bayes Classifier

In [37]:
from sklearn.naive_bayes import MultinomialNB

# Initialize Multinomial Naive Bayes model
model = MultinomialNB()

# Train (fit) the model
model.fit(X_train, y_train)

print("Naive Bayes model trained successfully!")


Naive Bayes model trained successfully!


7) Make Predictions

In [38]:
# Predict labels for test data
y_pred = model.predict(X_test)

print("Predictions completed!")
print("Sample Predictions:")
print(y_pred[:10])   # show first 10 predictions

Predictions completed!
Sample Predictions:
['ham' 'ham' 'ham' 'spam' 'ham' 'ham' 'ham' 'ham' 'ham' 'ham']


8) Evaluate Model

In [39]:
from sklearn.metrics import accuracy_score, classification_report

# Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Classification Report
report = classification_report(y_test, y_pred)

print("Model Evaluation Results")
print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)


Model Evaluation Results
Accuracy: 0.9659192825112107

Classification Report:
               precision    recall  f1-score   support

         ham       0.96      1.00      0.98       966
        spam       1.00      0.74      0.85       149

    accuracy                           0.97      1115
   macro avg       0.98      0.87      0.92      1115
weighted avg       0.97      0.97      0.96      1115



9) Compare with Logistic Regression

In [40]:
from sklearn.linear_model import LogisticRegression

# Initialize Logistic Regression model
log_reg = LogisticRegression(max_iter=1000)

# Train model
log_reg.fit(X_train, y_train)

# Predictions
y_pred_lr = log_reg.predict(X_test)

# Evaluation
accuracy_lr = accuracy_score(y_test, y_pred_lr)
report_lr = classification_report(y_test, y_pred_lr)

print("Logistic Regression Evaluation")
print("Accuracy:", accuracy_lr)
print("\nClassification Report:\n", report_lr)

Logistic Regression Evaluation
Accuracy: 0.957847533632287

Classification Report:
               precision    recall  f1-score   support

         ham       0.95      1.00      0.98       966
        spam       0.99      0.69      0.81       149

    accuracy                           0.96      1115
   macro avg       0.97      0.85      0.90      1115
weighted avg       0.96      0.96      0.95      1115

